In [1]:
import pandas as pd
import numpy as np
from collections import Counter
pd.options.display.max_rows = 200
pd.options.display.max_columns = 200
import warnings
warnings.filterwarnings("ignore")

def averageWithNumbers(arr, column):
    temp = []
    for i in range(0,len(arr),6):
        for j in range(0,6):
            if np.isnan(arr[i+j][column])==0:
                temp.append(arr[i+j][column])
        average = np.mean(temp)
        for k in range(0,6):
            if np.isnan(arr[i+k][column])==1:
                arr[i+k][column] = average
        temp.clear()
    return arr

def avNumberAllNan(arr,column):
    temp=[]
    for i in range(len(arr)):
        if arr[i][column] != 0 and np.isnan(arr[i][column])==False:
            temp.append(arr[i][column])
    average = np.mean(temp)
    for i in range(len(arr)):
        if np.isnan(arr[i][column]) == True:
            arr[i][column] = average
    return arr

def averageWithCategorial(arr, column):
    temp = []
    for i in range(0,len(arr),6):
        for j in range(0,6):
            if np.isnan(arr[i+j][column])==0:
                temp.append(arr[i+j][column])
        average = round(np.mean(temp),0)
        for k in range(0,6):
            if np.isnan(arr[i+k][column])==1:
                arr[i+k][column] = average
        temp.clear()
    return arr


def avCategorialAllNan(arr,column):
    temp=[]
    for i in range(len(arr)):
        if arr[i][column] != 0 and np.isnan(arr[i][column])==False:
            temp.append(arr[i][column])
    average = Counter(temp).most_common(1)[0][0]
    for i in range(len(arr)):
        if np.isnan(arr[i][column]) == True:
            arr[i][column] = average
    return arr

def avNumbers(train_arr):
    for i in range(16,20):
        train_arr = averageWithNumbers(train_arr,i)
        train_arr = avNumberAllNan(train_arr,i)
    for i in range(21,60):
        train_arr = averageWithNumbers(train_arr,i)
        train_arr = avNumberAllNan(train_arr,i)
    for i in range(61,65):
        train_arr = averageWithNumbers(train_arr,i)
        train_arr = avNumberAllNan(train_arr,i)
    for i in range(66,70):
        train_arr = averageWithNumbers(train_arr,i)
        train_arr = avNumberAllNan(train_arr,i)
    for i in range(72,82):
        train_arr = averageWithNumbers(train_arr,i)
        train_arr = avNumberAllNan(train_arr,i)

    train_arr = averageWithNumbers(train_arr,64)
    train_arr = avNumberAllNan(train_arr,64)
    train_arr = averageWithNumbers(train_arr,66)
    train_arr = avNumberAllNan(train_arr,66)
    
    
    train_arr = averageWithCategorial(train_arr,15)#Тип скважины
    train_arr = avCategorialAllNan(train_arr,15)
    train_arr = averageWithCategorial(train_arr,3)#Метод
    train_arr = averageWithCategorial(train_arr,4)#Характер работы
    train_arr = binarization(train_arr,20)#Тип скважины
    train_arr = binarization(train_arr,70)#Тип ГЗУ
    train_arr = binarization(train_arr,71)#ДНС
    train_arr = binarization(train_arr,60)#Назначение по проекту!!!!
    train_arr = binarization(train_arr,65)#Тип доп оборудования
    
    return train_arr

def binarization(arr,column):
    temp = []
    for i in range(0,len(arr),6):
        for j in range(0,6):
            if np.isnan(arr[i+j][column])==0:
                temp.append(arr[i+j][column])
        if len(temp)!=0:
            for k in range(0,6):
                arr[i+k][column] = temp[0]
        else:
            for k in range(0,6):
                arr[i+k][column] = 0
        temp.clear()
    return arr

#Загружаю датасеты
train_data = pd.read_csv('train_1.8.csv',encoding='cp1251').apply(lambda x: x.replace(',','.'))
test_data = pd.read_csv('test_1.9.csv',encoding='cp1251').apply(lambda x: x.replace(',','.'))
train_data = pd.DataFrame(data=train_data)
test_data = pd.DataFrame(data=test_data)

#Строки с НаНами в Таргете т.к. некоторые скважины с ними "В работе", а некоторые нет и хз че там должно быть ваще
train_data = train_data.dropna(subset=["Нефть, т"])

#Удаляем признаки которых нет в тесте
train_data = train_data.drop(["Нефть, м3", "Жидкость, м3", "Дебит жидкости", "ТП(ИДН) Дебит жидкости",
                             "ТП(ИДН) Дебит жидкости скорр-ый", "ТП(ГРП) Дебит жидкости", "ТП(ГРП) Дебит жидкости скорр-ый",
                             "ГП - Общий прирост Qн"], axis =1)
    
#Удаляю признаки, доверяясь аналитике от SWhite
train_data = train_data.drop(["Закачка, м3","Природный газ, м3", "Газ из газовой шапки, м3", "Конденсат, т"
                              , "Приемистость, м3/сут", "Тип газосепаратора","Агент закачки", "Дебит конденсата"
                              ,"КН закрепленный","Станок-качалка","Пласт МЭР","Куст","Диаметр экспл.колонны",
                              "SKIN","Дата ГРП","Фирма ГРП","Причина простоя.1","Примечание","Мероприятия"
                              ,"Диаметр дополнительного оборудования","Глубина спуска доп. оборудования"],axis=1)
    
#Удаляю признаки где только 0 и NaN. Оставляю один из них - число качаний ШГН
train_data = train_data.drop(["Длина хода плунжера ШГН","Диаметр плунжера","Коэффициент подачи насоса"],axis =1)

train_data = train_data.drop(["КНС","Число качаний ШГН","Марка ПЭД","Группа фонда","Неустановившийся режим"
                            ,"Нерентабельная","Фонтан через насос", "Способ эксплуатации"
                            , "Состояние на конец месяца"],axis = 1)

#Удаляю скважины с менее чем 6 измерениями
train_data = train_data.drop(train_data[train_data.Скважина == '08e65d04de1ab15b06df84960edd6d5a'].index)
train_data = train_data.drop(train_data[train_data.Скважина == '887d2c82cde2f458d2d6f44f9f38e26d'].index)
train_data = train_data.drop(train_data[train_data.Скважина == '8c13009d0a7b6ecb7768179fc5d6d421'].index)
train_data = train_data.drop(train_data[train_data.Скважина == '8efb4a39dfc57f8d8e4b537b11d0686a'].index)
train_data = train_data.drop(train_data[train_data.Скважина == 'daf2ea4b3da3a5d883c189bec839993a'].index)
train_data = train_data.drop(train_data[train_data.Скважина == 'c3ea8af72e24d55ab2fb0fad507b3a70'].index)
train_data = train_data.drop(train_data[train_data.Скважина == '7f7ffc6b5c3a8a81fd4c6a30b1ad41a6'].index)
train_data = train_data.drop(train_data[train_data.Скважина == '68d84139c0c884fde6b803f8b3eca712'].index)
train_data = train_data.drop(train_data[train_data.Скважина == '5fcdb7a8e9a31fd603ad3e5fac59d0db'].index)
train_data = train_data.drop(train_data[train_data.Скважина == '421f1a8c797cf8a8af6b656e637a8623'].index)
train_data = train_data.drop(train_data[train_data.Скважина == 'b6870b69eec1a287a0f2ba99183e5e42'].index)
train_data = train_data.drop(train_data[train_data.Скважина == 'da6adb51ff45fe0fc91e316bb62f0a90'].index)
train_data = train_data.drop(train_data[train_data.Скважина == '941826ec04693beff13830c95c1c50ba'].index)
train_data = train_data.drop(train_data[train_data.Скважина == 'a2fc99d445adc1de96a7f467de2070c8'].index)
train_data = train_data.drop(train_data[train_data.Скважина == '2f80cdbb4d9d96f34f94a3bc64fa17af'].index)
train_data = train_data.drop(train_data[train_data.Скважина == '4e18442617fff3921f583333c8e91029'].index)
train_data = train_data.drop(train_data[train_data.Скважина == '8f3131afe66d3266c23142786786b265'].index)
train_data = train_data.drop(train_data[train_data.Скважина == '79c35740ca159eb6f85b880c90bbd84f'].index)
train_data = train_data.drop(train_data[train_data.Скважина == '38ffaacf7b8d60b68d9c7044141e54bc'].index)
train_data = train_data.drop(train_data[train_data.Скважина == 'da18db02993aa8c772150458a6364a33'].index)
train_data = train_data.drop(train_data[train_data.Скважина == '5d995ad52dbe6eff8f1b6681d6b21bec'].index)
train_data = train_data.drop(train_data[train_data.Скважина == '15a7b4ca85115874d8af73746b9c3c48'].index)
train_data = train_data.drop(train_data[train_data.Скважина == 'f6a7b0e5d0ff95d68c72b28b6ac2b93b'].index)
train_data = train_data.drop(train_data[train_data.Скважина == 'bd2a090a38f039a8a2a93e2112f31042'].index)



#КНС,Число качаний ШГН, Марка ПЭД,Группа фонда,
#неустановившийся режим, неретнабельная, фонтан через насос,способ эксплуатации,
#состояние на конец месяца


train_data["Причина простоя"] = train_data["Причина простоя"].fillna(value= "Нет_Причины")

    
map = {'Наклонно-направленно':1,'Горизонтально':2,'Вертикально':2}
train_data = train_data.replace({'Тип скважины':map})
map = {'()':1,'Массомер ()':2,'ГЗУ "Спутник" ()':2}
train_data = train_data.replace({'Тип ГЗУ':map})
map = {'ДНС-1':1,'ДНС-3':2,'ДНС-5':3,'ДНС-2':4,'ДНС-7':4}
train_data = train_data.replace({'ДНС':map})
map = {'Хвостовик':1}
train_data = train_data.replace({'Тип дополнительного оборудования':map})
map = {'Нефтяные':1,'Нагнетательные':2}
train_data = train_data.replace({'Назначение по проекту':map})
map = {'FC-925(125)-2400':1,'ЭЦНКИД 5a-125-2400':2,'ЭЦНКИД 5a-125-2500':3,'FC-450(50)-2400':4
       ,'FC-650(80)-2400':5,'ЭЦНКИД 5-80-2500':6,'FC-650(80)-2300':7,'FC-925(125)-2300':8
       ,'D700EZ-2450':9,'D420EZ-2400':10,'FC-925-2500':11,'FC Р-3-2400':12,'ВНН5-44-2400':13,'DN-1100-2300':13
       ,'ЭЦНКИД 5-50-2500':14,'ВНН-79-2400':15,'ЭЦН-60-2500':16,'FC-925-2450':16,'D420EZ-2200':16
       ,'FC P-6-2500':16,'ЭЦНКИД 5-50-2300':16,'ЭЦНКИД 5а-125-2600':16,'D530EZ-2550':16,'467 MT100':17
       ,'FC-450(50)-2300':17,'ВНН-59-2350':17,'ЭЦНКИД 5-80-2300':17,'D700EZ-2250':17,'ЭЦН-35-2500':17
       ,'ВНН5-59-2450':17,'ВНН5-79-2600':18,'FC Р-10-2400':18,'DN-1100-2300 ':18,'ЭЦНД-100-2400':18
       ,'FC-925(125)-2250':18,'ЭЦН-50-2700':18,'D1150N':18,'ВНН5-79-2400':18,'ВНН-100-2400':18,'FC P-6-2400':18
       ,'ЭЦН (МТ)-80-2400':18,'FC-650(80)-2250':19,'ЭЦНКИД 5a-100-2400':19,'ЭЦН-60-2400':19,'ВНН5-59-2250':19
       ,'ЭЦН-80-2900':19,'DN-1100-2250':19,'ЭЦН (МТ)-80-2350':19,'ЭЦНД-60-2200':19,'ЭЦН-60-2800':19
       ,'ВНН5-79-2350':19,'FC-1600(200)-2400':20,'ЭЦН-35-3150':20,'FC Р-3-2300':20,'D700EZ-2150':20
       ,'ЭЦН-35-2900':20,'D1000EZ-2500':20,'FC-650(80)-2500':20,'ВНН5-59-2350':20,'ВНН5-79-2500':20
       ,'FC P-10-2450':20,'ВНН5-125-2600':20,'380 D1200EZ':20,'ВНН-59-2200':20,'FC P-12-2500':21,'D1000EZ-2300':21
       ,'ВНН5-124-2500':21,'ВНН-44-2450':21,'D420EZ-2500':21,'ЭЦНКИД 5-80-2400':21,'ВНН-124-2300':21
       ,'ЭЦНД-125-2400':21,'FC-450(50)-2350':21,'ВНН-100-2250':21,'FC P-4-2450':21,'ВНН-79-2300':21
       ,'D1000EZ-2400':21,'D700EZ-2400':21,'Воронка с пакером':21,'ВНН-59-2250':22,'ВНН5-125-2350 Новомет':22
       ,'ЭЦНКИД 5-80-2600':22,'D530EZ-2700':22,'ЭЦН-100-2400':22,'ВНН5-59-2400':22,'D1000EZ-2200':22
       ,'FC-230-2400':22,'FC-650(80)-2450':22,'ЭЦН-60-2350':22,'D700EZ-2300':22,'SPI D-20-2350':22
       ,'FC-925(125)-2350':22,'ЭЦН (МТ)-60-2350':22,'ВНН5-59-2600':23,'Х-10(125)-2400':23,'ЭЦН-125-2600':23
       ,'ЭЦНД-100-2350':23,'ВНН-79-2350':23,'D420EZ-2300':23,'FC P6-2550':23,'ЭЦН (МТ)-80-2500':23
       ,'ВНН-59-2400':23}
train_data = train_data.replace({'Тип насоса':map})
map = {'ЭЦН':1,'ЭЦН/ФОН':2,'ФОН':3,'ЭЦН/ -':3,'ФОН/ЭЦН':3}
train_data = train_data.replace({'Метод':map})
map = {'НЕФ':1,'НАГ':2,'НЕФ/НАГ':3}
train_data = train_data.replace({'Характер работы':map})
map = {'Нет_Причины':0,'Геологич.причина':1,'Прочие':2,'Прочие aвaрии':2,'ПРИНЯТА В ОСВОЕНИЕ':3
       ,'Принята в освоение':3,'Останов по распоряжен.':4,'Техническ.причина':5,'R=0':6,'Ввод новой скважины':7
       ,'Перевод в ППД':8,'ПЕРЕВОД В ППД':8,'Нет подачи':9,'Освоение после бурения':10,'ОСВОЕНИЕ ПОСЛЕ БУРЕНИЯ':10
       ,'ГРП':11,'Заклинивание насос':12,'Перевод скв.в другой фонд':13,'Ремонт назем.электрообор.':14
       ,'Ревиз.подзем.оборудования':15,'Отключение напряжения':15,'Отсутствие связи с ТМС':15
       ,'НЕТ ПРИЕМИСТОСТИ':15,'Ремонт соседней скважина':15,'СКО пласта':15,'НЕИСПРАВНОСТЬ СУ, ЧП':15
       ,'Заморожен коллектор':15,'Недоосвоена после ПРС':15,'Нет приемистости':15,'Замена СУ на ЧП':15
       ,'Пропaркa фонтaн.aрмaтуры':15,'Ремонт запорной арматуры':15,'Дисбаланс напряжения':15
       ,'Сменa НКТ из-зa негермет.':15,'ОТКЛЮЧЕНИЕ ФИДЕРА':15,'Недогрузка':15,'Неисправ.глуб.оборуд.':15
       ,'Технолог.причина':15,'ОЖИДАН.КАПИТАЛ.РЕМОНТА':15}
train_data = train_data.replace({'Причина простоя':map})
map = {'РАБ.':1,'ОСВ ТГ':2,'НАК':3,'ОСТ.':4,'Б/Д ТГ':5,'ПЬЕЗ':6,'ОЖ ОСВ ТГ':7,'ОСВ ПР ЛЕТ':8,'Б/Д ПР Л':9}
train_data = train_data.replace({'Состояние':map})

train_arr = train_data.values
train_arr = avNumbers(train_arr)
train_data = pd.DataFrame(data = train_arr)

train_data[3] = train_data[3].fillna(value= 1)
train_data[4] = train_data[4].fillna(value= 1)
train_data[5] = train_data[5].fillna(value= 0)

train_data.to_csv('gazporn.csv')

train_data = pd.read_csv('gazporn.csv',encoding='cp1251',names=['Скважина','Дата','ГТМ','Метод','Характер работы'
                                                      ,'Состояние','Время работы, ч','Время накопления'
                                                      ,'Нефть, т','Попутный газ, м3','Простой, ч','Причина простоя'
                                                      ,'Обводненность (вес), %','Добыча растворенного газа, м3'
                                                      ,'Дебит попутного газа, м3/сут','Тип скважины','Диаметр НКТ'
                                                      ,'Диаметр штуцера','Глубина верхних дыр перфорации','Удлинение'
                                                      ,'Тип насоса','Производительность ЭЦН','Напор','Частота'
                                                      ,'Коэффициент сепарации','Глубина спуска','Буферное давление'
                                                      ,'Давление в линии','Пластовое давление','Динамическая высота'
                                                      ,'Затрубное давление','Давление на приеме','Забойное давление'
                                                      ,'Обводненность','Давление наcыщения','Газовый фактор'
                                                      ,'Температура пласта','JD факт','Вязкость нефти в пластовых условиях'
                                                      ,'Вязкость воды в пластовых условиях','Вязкость жидкости в пласт. условиях'
                                                      ,'объемный коэффициент','Плотность нефти','Плотность воды'
                                                      ,'Высота перфорации','Удельный коэффициент','Коэффициент продуктивности'
                                                      ,'ТП - Забойное давление','ТП - JD опт.','ТП - SKIN','К пр от стимуляции'
                                                      ,'Глубина спуска.1','КВЧ','Время до псевдоуст-ся режима'
                                                      ,'Дата запуска после КРС','Дата пуска','Дата останова'
                                                      ,'Радиус контура питания','Номер бригады','Дата ввода в эксплуатацию'
                                                      ,'Назначение по проекту','Замерное забойное давление'
                                                      ,'Нефтенасыщенная толщина','Плотность раствора глушения'
                                                      ,'Глубина текущего забоя','Тип дополнительного оборудования'
                                                      ,'Мощность ПЭД','I X/X','Ток номинальный','Ток рабочий','Тип ГЗУ','ДНС'
                                                      ,'Пластовое давление начальное','Характеристический дебит жидкости'
                                                      ,'Время в работе','Время в накоплении','ГП - Забойное давление'
                                                      ,'ГП(ИДН) Дебит жидкости','ГП(ИДН) Дебит жидкости скорр-ый'
                                                      ,'ГП(ИДН) Прирост дефита нефти','ГП(ГРП) Дебит жидкости'
                                                      ,'ГП(ГРП) Дебит жидкости скорр-ый'])
    
train_data = train_data[0:]
train_data.to_csv('gazporn.csv',encoding='cp1251')

In [2]:
#Дропаем левую колонку, дату (потом че то придумать) и ИДшник скважины
data = pd.read_csv('gazporn.csv',encoding='cp1251',decimal=',').drop(['Unnamed: 0','Скважина','Дата','Характер работы',
                    'Время накопления','Добыча растворенного газа, м3','Тип скважины','Тип насоса',
                    'Коэффициент сепарации',
                    'Вязкость воды в пластовых условиях',
                    'Плотность нефти',
                    'Плотность воды',
                    'ТП - JD опт.',
                    'ТП - SKIN',
                    'Радиус контура питания',
                    'Назначение по проекту',
                    'Замерное забойное давление',
                    'Тип дополнительного оборудования',
                    'Мощность ПЭД',
                    'I X/X',
                    'Ток номинальный',
                    'Тип ГЗУ',
                    'ДНС',
                    'Время в работе',
                    'Время в накоплении',
                    'ГП(ИДН) Дебит жидкости скорр-ый',
                    'ГП(ГРП) Дебит жидкости скорр-ый'
],axis=1).apply(lambda x: x.replace(',','.'))


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error
import xgboost as xgb

In [4]:
y = np.array(data['Нефть, т'])
data = data.drop(['Нефть, т'], axis=1)
X = np.array(data)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [6]:
model = xgb.XGBRegressor()
model.fit(X_train,y_train,eval_metric='mae')
y_pred_xgb = model.predict(X_test)
print(mean_absolute_error(y_test,y_pred_xgb))

111.62078248435597


In [28]:
data

,ГТМ,Метод,Состояние,"Время работы, ч","Попутный газ, м3","Простой, ч",Причина простоя,"Обводненность (вес), %","Дебит попутного газа, м3/сут",Диаметр НКТ,Диаметр штуцера,Глубина верхних дыр перфорации,Удлинение,Производительность ЭЦН,Напор,Частота,Глубина спуска,Буферное давление,Давление в линии,Пластовое давление,Динамическая высота,Затрубное давление,Давление на приеме,Забойное давление,Обводненность,Давление наcыщения,Газовый фактор,Температура пласта,JD факт,Вязкость нефти в пластовых условиях,Вязкость жидкости в пласт. условиях,объемный коэффициент,Высота перфорации,Удельный коэффициент,Коэффициент продуктивности,ТП - Забойное давление,К пр от стимуляции,Глубина спуска.1,КВЧ,Время до псевдоуст-ся режима,Дата запуска после КРС,Дата пуска,Дата останова,Номер бригады,Дата ввода в эксплуатацию,Нефтенасыщенная толщина,Плотность раствора глушения,Глубина текущего забоя,Ток рабочий,Пластовое давление начальное,Характеристический дебит жидкости,ГП - Забойное давление,ГП(ИДН) Дебит жидкости,ГП(ИДН) Прирост дефита нефти,ГП(ГРП) Дебит жидкости
0,0.0,2.0,1.0,301.0,10028.0,0.0,0,69.7,799.54,67.4,0.0,2387.0,34.84,60.0,2460.0,53.7,2311.0,27.0,26.0,168.33999999999997,2113.4,25.9,41.4,47.873747831509704,37.2,80.9,36.0,87.0,0.1348204654256092,1.67,1.6687585901340882,1.12,13.0,9.128395335949758,0.4878221138020114,39.68202089668918,1.979626107064568,0.0,67.6,23.843339970838343,42143.4444444444,42143.4444444444,42118.0,0.0,42135.0,6.0,0.7514667369566701,2458.0,211.0,231.0,637.217130812687,36.0,61.44665296530967,1.8944140243939145,248.15843481372383
1,0.0,1.0,1.0,720.0,38690.0,0.0,0,38.2,1289.68,67.4,0.0,2387.0,34.84,60.0,2460.0,45.0,2311.0,26.0,25.0,190.0,1720.0,16.6,66.0,72.773373753411,66.0,80.9,36.0,87.0,0.141289807627567,1.67,1.66779850597212,1.12,13.0,12.138039082360196,0.6489998216651421,40.0,2.4461437268582795,0.0,82.0,17.4426364023932,42143.4444444444,42143.4444444444,42118.0,0.0,42135.0,6.0,0.848156587987212,2458.0,211.0,231.0,847.308442842446,35.0,96.49325140949672,6.06190376692914,363.69249072195396
2,0.0,1.0,1.0,744.0,37685.0,0.0,0,35.1,1215.64,67.4,0.0,2387.0,34.84,60.0,2460.0,55.0,2311.0,27.0,26.0,190.0,2164.0,27.2,39.0,45.45156461328239,36.0,80.9,36.0,87.0,0.136382993824908,1.67,1.6687984651051202,1.12,13.0,8.45861552466757,0.451996445875633,40.0,1.87728197034192,0.0,64.0,25.045037450665802,42143.4444444444,42143.4444444444,42118.0,0.0,42135.0,6.0,0.848156587987212,2458.0,211.0,231.0,590.462454452354,36.0,65.6651581996136,1.4839600855448798,272.72784707183996
3,0.0,1.0,1.0,744.0,32434.0,0.0,0,32.1,1046.26,67.4,0.0,2387.0,34.84,60.0,2460.0,56.0,2311.0,26.5,25.4,153.9,2255.0,27.3,32.0,38.4101044834459,30.0,80.9,36.0,87.0,0.13148329172216802,1.67,1.6689986008395001,1.12,13.0,9.23944201786856,0.49366165880720203,38.4101044834459,2.00097153658728,0.0,64.0,22.9312277195588,42143.4444444444,42143.4444444444,42118.0,0.0,42135.0,6.0,0.6870068362696421,2458.0,211.0,231.0,644.968859942986,35.4,53.8386482491881,0.5107367837555851,218.225581819052
4,0.0,1.0,1.0,720.0,27422.0,0.0,0,34.2,914.06,67.4,0.0,2387.0,34.84,60.0,2460.0,56.0,2311.0,28.5,27.6,153.9,2230.0,28.4,35.0,41.344787130140006,25.0,80.9,36.0,87.0,0.13231200429558,1.67,1.6691654172913502,1.12,13.0,8.35527662513608,0.4463762519362261,40.0,1.8602984097478201,0.0,64.0,25.3603722541683,42143.4444444444,42143.4444444444,42118.0,0.0,42135.0,6.0,0.6870068362696421,2458.0,211.0,231.0,583.2487750884179,37.6,47.9817368233338,0.640583277225278,199.966616329175
5,0.0,1.0,1.0,744.0,23469.0,0.0,0,38.5,757.05,67.4,0.0,2387.0,34.84,60.0,2460.0,56.5,2311.0,27.0,26.0,153.9,2198.0,30.0,35.0,41.3889091772692,29.0,80.9,36.0,87.0,0.132634229657823,1.67,1.66903196146235,1.12,13.0,7.450603429716389,0.3980763907258541,40.0,1.71343489178754,0.0,64.0,28.4374260274056,42143.4444444444,42143.4444444444,42118.0,0.0,42135.0,6.0,0.6870068362696421,2458.0,211.0,231.0,520.097121737231,36.0,43.254470144916105,0.7748862085146908,186.179638126598
6,0.0,2.0,1.0,111.0,13993.0,0.0,7,33.7,3025.52,73.0,32.0,2817.13,461

In [22]:
for i in list(zip(data.columns.get_values(),model.feature_importances_)):
    if i[1]==0:
        print("'"+i[0]+"',")

'Характер работы',
'Время накопления',
'Добыча растворенного газа, м3',
'Тип скважины',
'Тип насоса',
'Коэффициент сепарации',
'Вязкость воды в пластовых условиях',
'Плотность нефти',
'Плотность воды',
'ТП - JD опт.',
'ТП - SKIN',
'Радиус контура питания',
'Назначение по проекту',
'Замерное забойное давление',
'Тип дополнительного оборудования',
'Мощность ПЭД',
'I X/X',
'Ток номинальный',
'Тип ГЗУ',
'ДНС',
'Время в работе',
'Время в накоплении',
'ГП(ИДН) Дебит жидкости скорр-ый',
'ГП(ГРП) Дебит жидкости скорр-ый',


In [54]:
import matplotlib.pyplot as plt

y = np.array(pd.read_csv('gazporn.csv',encoding='cp1251',decimal=',')['Нефть, т'])

lists = []
for i in range(0,len(y),6):
    list_ = []
    for j in range(6):
        list_.append(y[i+j])
    list_ = np.array(list_)
    #list_ = np.interp(list_, (list_.min(), list_.max()), (1, 100))
    lists.append(list_)

x = np.arange(6)
    
for l in lists:
    plt.plot(x, l)
plt.show()

<Figure size 640x480 with 1 Axes>

In [55]:
lists = np.array(lists)
len(lists[:,1])
print(np.mean(lists[:,5]))

597.105456475584


In [40]:
test_data = pd.read_csv('test_1.9.csv',encoding='cp1251',decimal=',').apply(lambda x: x.replace(',','.'))
test_data = test_data[data.columns.get_values()]

In [41]:
test_data["Причина простоя"] = test_data["Причина простоя"].fillna(value= "Нет_Причины")

    
map = {'Наклонно-направленно':1,'Горизонтально':2,'Вертикально':2}
test_data = test_data.replace({'Тип скважины':map})
map = {'()':1,'Массомер ()':2,'ГЗУ "Спутник" ()':2}
test_data = test_data.replace({'Тип ГЗУ':map})
map = {'ДНС-1':1,'ДНС-3':2,'ДНС-5':3,'ДНС-2':4,'ДНС-7':4}
test_data = test_data.replace({'ДНС':map})
map = {'Хвостовик':1}
test_data = test_data.replace({'Тип дополнительного оборудования':map})
map = {'Нефтяные':1,'Нагнетательные':2}
test_data = test_data.replace({'Назначение по проекту':map})
map = {'FC-925(125)-2400':1,'ЭЦНКИД 5a-125-2400':2,'ЭЦНКИД 5a-125-2500':3,'FC-450(50)-2400':4
       ,'FC-650(80)-2400':5,'ЭЦНКИД 5-80-2500':6,'FC-650(80)-2300':7,'FC-925(125)-2300':8
       ,'D700EZ-2450':9,'D420EZ-2400':10,'FC-925-2500':11,'FC Р-3-2400':12,'ВНН5-44-2400':13,'DN-1100-2300':13
       ,'ЭЦНКИД 5-50-2500':14,'ВНН-79-2400':15,'ЭЦН-60-2500':16,'FC-925-2450':16,'D420EZ-2200':16
       ,'FC P-6-2500':16,'ЭЦНКИД 5-50-2300':16,'ЭЦНКИД 5а-125-2600':16,'D530EZ-2550':16,'467 MT100':17
       ,'FC-450(50)-2300':17,'ВНН-59-2350':17,'ЭЦНКИД 5-80-2300':17,'D700EZ-2250':17,'ЭЦН-35-2500':17
       ,'ВНН5-59-2450':17,'ВНН5-79-2600':18,'FC Р-10-2400':18,'DN-1100-2300 ':18,'ЭЦНД-100-2400':18
       ,'FC-925(125)-2250':18,'ЭЦН-50-2700':18,'D1150N':18,'ВНН5-79-2400':18,'ВНН-100-2400':18,'FC P-6-2400':18
       ,'ЭЦН (МТ)-80-2400':18,'FC-650(80)-2250':19,'ЭЦНКИД 5a-100-2400':19,'ЭЦН-60-2400':19,'ВНН5-59-2250':19
       ,'ЭЦН-80-2900':19,'DN-1100-2250':19,'ЭЦН (МТ)-80-2350':19,'ЭЦНД-60-2200':19,'ЭЦН-60-2800':19
       ,'ВНН5-79-2350':19,'FC-1600(200)-2400':20,'ЭЦН-35-3150':20,'FC Р-3-2300':20,'D700EZ-2150':20
       ,'ЭЦН-35-2900':20,'D1000EZ-2500':20,'FC-650(80)-2500':20,'ВНН5-59-2350':20,'ВНН5-79-2500':20
       ,'FC P-10-2450':20,'ВНН5-125-2600':20,'380 D1200EZ':20,'ВНН-59-2200':20,'FC P-12-2500':21,'D1000EZ-2300':21
       ,'ВНН5-124-2500':21,'ВНН-44-2450':21,'D420EZ-2500':21,'ЭЦНКИД 5-80-2400':21,'ВНН-124-2300':21
       ,'ЭЦНД-125-2400':21,'FC-450(50)-2350':21,'ВНН-100-2250':21,'FC P-4-2450':21,'ВНН-79-2300':21
       ,'D1000EZ-2400':21,'D700EZ-2400':21,'Воронка с пакером':21,'ВНН-59-2250':22,'ВНН5-125-2350 Новомет':22
       ,'ЭЦНКИД 5-80-2600':22,'D530EZ-2700':22,'ЭЦН-100-2400':22,'ВНН5-59-2400':22,'D1000EZ-2200':22
       ,'FC-230-2400':22,'FC-650(80)-2450':22,'ЭЦН-60-2350':22,'D700EZ-2300':22,'SPI D-20-2350':22
       ,'FC-925(125)-2350':22,'ЭЦН (МТ)-60-2350':22,'ВНН5-59-2600':23,'Х-10(125)-2400':23,'ЭЦН-125-2600':23
       ,'ЭЦНД-100-2350':23,'ВНН-79-2350':23,'D420EZ-2300':23,'FC P6-2550':23,'ЭЦН (МТ)-80-2500':23
       ,'ВНН-59-2400':23}
test_data = test_data.replace({'Тип насоса':map})
map = {'ЭЦН':1,'ЭЦН/ФОН':2,'ФОН':3,'ЭЦН/ -':3,'ФОН/ЭЦН':3,'ЭЦН/Без способа':3}
test_data = test_data.replace({'Метод':map})
map = {'НЕФ':1,'НАГ':2,'НЕФ/НАГ':3}
test_data = test_data.replace({'Характер работы':map})
map = {'Нет_Причины':0,'Геологич.причина':1,'Прочие':2,'Прочие aвaрии':2,'ПРИНЯТА В ОСВОЕНИЕ':3
       ,'Принята в освоение':3,'Останов по распоряжен.':4,'Техническ.причина':5,'R=0':6,'Ввод новой скважины':7
       ,'Перевод в ППД':8,'ПЕРЕВОД В ППД':8,'Нет подачи':9,'Освоение после бурения':10,'ОСВОЕНИЕ ПОСЛЕ БУРЕНИЯ':10
       ,'ГРП':11,'Заклинивание насос':12,'Перевод скв.в другой фонд':13,'Ремонт назем.электрообор.':14
       ,'Ревиз.подзем.оборудования':15,'Отключение напряжения':15,'Отсутствие связи с ТМС':15
       ,'НЕТ ПРИЕМИСТОСТИ':15,'Ремонт соседней скважина':15,'СКО пласта':15,'НЕИСПРАВНОСТЬ СУ, ЧП':15
       ,'Заморожен коллектор':15,'Недоосвоена после ПРС':15,'Нет приемистости':15,'Замена СУ на ЧП':15
       ,'Пропaркa фонтaн.aрмaтуры':15,'Ремонт запорной арматуры':15,'Дисбаланс напряжения':15
       ,'Сменa НКТ из-зa негермет.':15,'ОТКЛЮЧЕНИЕ ФИДЕРА':15,'Недогрузка':15,'Неисправ.глуб.оборуд.':15
       ,'Технолог.причина':15,'ОЖИДАН.КАПИТАЛ.РЕМОНТА':15,'Останов скв - исслед., замеры':15
       ,'ОТКЛЮЧЕНИЕ НАПРЯЖЕНИЯ':15,'ПЕРЕВ.НA МЕXДОБ.ПРЕК.ФОН.':15,'Перевод нa ЭЦН':15,'Высокое напряжение':15}
test_data = test_data.replace({'Причина простоя':map})
map = {'РАБ.':1,'ОСВ ТГ':2,'НАК':3,'ОСТ.':4,'Б/Д ТГ':5,'ПЬЕЗ':6,'ОЖ ОСВ ТГ':7,'ОСВ ПР ЛЕТ':8,'Б/Д ПР Л':9}
test_data = test_data.replace({'Состояние':map})

In [42]:

data_1st_month = []
for i in range(0,len(data),6):
    data_1st_month.append(np.array(data.iloc[i]))

data_1st_month = pd.DataFrame(data_1st_month,columns=data.columns.values)
#TODO: сделать датафрейм DATA_N_Month по всем месяцам и дополнять тест соответствующе

In [43]:

#НАНы в категориальных признаках меняем на самый популярный из них
categorical_features = {'ГТМ':data['ГТМ'].value_counts()[0],'Метод':data['Метод'].value_counts()[0],
                        'Состояние':data['Состояние'].value_counts()[0],
                        'Причина простоя':data['Причина простоя'].value_counts()[0],
                        }

#НАНы в статических меняем на среднее из трейна
static_features = {'Диаметр НКТ':data_1st_month['Диаметр НКТ'].astype(float).median(),
                   'Диаметр штуцера':data_1st_month['Диаметр штуцера'].astype(float).median(),
                   'Глубина верхних дыр перфорации':data_1st_month['Глубина верхних дыр перфорации'].astype(float).median(),
                   'Удлинение':data_1st_month['Удлинение'].astype(float).median(),
                   'Производительность ЭЦН':data_1st_month['Производительность ЭЦН'].astype(float).median(),
                   'Напор':data_1st_month['Напор'].astype(float).median(),
                   'Глубина спуска':data_1st_month['Глубина спуска'].astype(float).median(),
                   'Давление наcыщения':data_1st_month['Давление наcыщения'].astype(float).median(),
                   'Газовый фактор':data_1st_month['Газовый фактор'].astype(float).median(),
                   'Температура пласта':data_1st_month['Температура пласта'].astype(float).median(),
                   'Вязкость нефти в пластовых условиях':data_1st_month['Вязкость нефти в пластовых условиях'].astype(float).median(),
                   'объемный коэффициент':data_1st_month['объемный коэффициент'].astype(float).median(),
                   'Высота перфорации':data_1st_month['Высота перфорации'].astype(float).median(),
                   'Глубина спуска.1':data_1st_month['Глубина спуска.1'].astype(float).median(),
                   'КВЧ':data_1st_month['КВЧ'].astype(float).median(),
                   'Дата запуска после КРС':data_1st_month['Дата запуска после КРС'].astype(float).median(),
                   'Дата пуска':data_1st_month['Дата пуска'].astype(float).median(),
                   'Дата останова':data_1st_month['Дата останова'].astype(float).median(),
                   'Номер бригады':data_1st_month['Номер бригады'].astype(float).median(),
                   'Дата ввода в эксплуатацию':data_1st_month['Дата ввода в эксплуатацию'].astype(float).median(),
                   'Нефтенасыщенная толщина':data_1st_month['Нефтенасыщенная толщина'].astype(float).median(),
                   'Глубина текущего забоя':data_1st_month['Глубина текущего забоя'].astype(float).median(),
                   'Ток рабочий':data_1st_month['Ток рабочий'].astype(float).median(),
                   'Пластовое давление начальное':data_1st_month['Пластовое давление начальное'].astype(float).median(),
                   'ГП - Забойное давление':data_1st_month['ГП - Забойное давление'].astype(float).median(),
                   #А тут начинается рандом
                   'Время работы, ч':data_1st_month['Время работы, ч'].astype(float).median(),
                   'Попутный газ, м3':data_1st_month['Попутный газ, м3'].astype(float).median(),
                   'Простой, ч':data_1st_month['Простой, ч'].astype(float).median(),
                   'Обводненность (вес), %':data_1st_month['Обводненность (вес), %'].astype(float).median(),
                   'Дебит попутного газа, м3/сут':data_1st_month['Дебит попутного газа, м3/сут'].astype(float).median(),
                   'Частота':data_1st_month['Частота'].astype(float).median(),
                   'Буферное давление':data_1st_month['Буферное давление'].astype(float).median(),
                   'Давление в линии':data_1st_month['Давление в линии'].astype(float).median(),
                   'Пластовое давление':data_1st_month['Пластовое давление'].astype(float).median(),
                   'Динамическая высота':data_1st_month['Динамическая высота'].astype(float).median(),
                   'Затрубное давление':data_1st_month['Затрубное давление'].astype(float).median(),
                   'Давление на приеме':data_1st_month['Давление на приеме'].astype(float).median(),
                   'Забойное давление':data_1st_month['Забойное давление'].astype(float).median(),
                   'Обводненность':data_1st_month['Обводненность'].astype(float).median(),
                   'JD факт':data_1st_month['JD факт'].astype(float).median(),
                   'Вязкость жидкости в пласт. условиях':data_1st_month['Вязкость жидкости в пласт. условиях'].astype(float).median(),
                   'Удельный коэффициент':data_1st_month['Удельный коэффициент'].astype(float).median(),
                   'Коэффициент продуктивности':data_1st_month['Коэффициент продуктивности'].astype(float).median(),
                   'ТП - Забойное давление':data_1st_month['ТП - Забойное давление'].astype(float).median(),
                   'К пр от стимуляции':data_1st_month['К пр от стимуляции'].astype(float).median(),
                   'Время до псевдоуст-ся режима':data_1st_month['Время до псевдоуст-ся режима'].astype(float).median(),
                   'Плотность раствора глушения':data_1st_month['Плотность раствора глушения'].astype(float).median(),
                   'Характеристический дебит жидкости':data_1st_month['Характеристический дебит жидкости'].astype(float).median(),
                   'ГП(ИДН) Дебит жидкости':data_1st_month['ГП(ИДН) Дебит жидкости'].astype(float).median(),
                   'ГП(ИДН) Прирост дефита нефти':data_1st_month['ГП(ИДН) Прирост дефита нефти'].astype(float).median(),
                   'ГП(ГРП) Дебит жидкости':data_1st_month['ГП(ГРП) Дебит жидкости'].astype(float).median()
                  }

In [44]:
for k,v in categorical_features.items():
    test_data[k].fillna(value=v,inplace=True)

for k,v in static_features.items():
    test_data[k].fillna(value=v,inplace=True)
    
test_data

,ГТМ,Метод,Состояние,"Время работы, ч","Попутный газ, м3","Простой, ч",Причина простоя,"Обводненность (вес), %","Дебит попутного газа, м3/сут",Диаметр НКТ,Диаметр штуцера,Глубина верхних дыр перфорации,Удлинение,Производительность ЭЦН,Напор,Частота,Глубина спуска,Буферное давление,Давление в линии,Пластовое давление,Динамическая высота,Затрубное давление,Давление на приеме,Забойное давление,Обводненность,Давление наcыщения,Газовый фактор,Температура пласта,JD факт,Вязкость нефти в пластовых условиях,Вязкость жидкости в пласт. условиях,объемный коэффициент,Высота перфорации,Удельный коэффициент,Коэффициент продуктивности,ТП - Забойное давление,К пр от стимуляции,Глубина спуска.1,КВЧ,Время до псевдоуст-ся режима,Дата запуска после КРС,Дата пуска,Дата останова,Номер бригады,Дата ввода в эксплуатацию,Нефтенасыщенная толщина,Плотность раствора глушения,Глубина текущего забоя,Ток рабочий,Пластовое давление начальное,Характеристический дебит жидкости,ГП - Забойное давление,ГП(ИДН) Дебит жидкости,ГП(ИДН) Прирост дефита нефти,ГП(ГРП) Дебит жидкости
0,1.0,3.0,2.0,0.0,0.0,0.0,0,0.0,0.00,66.7807,27.1589,3105.69,479.931,99.0149,2424.17,50.0487,2853.19,20.1894,18.8149,262.913,2323.17,19.58,55.6293,71.9316,23.3886,99.9093,70.7424,92.0443,0.322212,1.33545,1.39247,1.16992,36.0957,3.74233,0.28292,48.222,0.650431,2782.74,124.753,1399.33,42118.2,42109.6,42069.7,2.47797,42078.6,13.3323,1.03842,3322.14,22.931,269.079,253.025,29.7971,58.9964,7.95461,127.841
1,2379,2201.0,2474.0,12,153,0,0,0.0,37.45,60.4,18.0,2617.0,109.59,50.0,2300.0,50.0,2482.0,25.0,24.0,0.0,2357.0,24.0,0.0,0.0,63.0,118.0,90.0,0.0,0.0,0.0,0.0,1.35,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,41455.0840277778,41455.0840277778,41448.0,1.0,41448.0,7.5,0.0,0.0,0.0,255.0,0.0,0.0,0.0,0.0,0.0
2,2379,2201.0,2474.0,12,153,0,0,0.0,37.45,68.3,0.0,3217.94,586.11,125.0,2600.0,49.0,2805.0,23.0,22.0,264.0,1551.0,23.3,113.0,119.955215306031,52.0,80.0,100.0,92.0,0.4214507527172921,1.38,1.5155508697944098,1.1440000000000001,5.45,12.5415960492241,0.902497096831321,59.52734375,0.0,2805.0,120.0,18.40839965854289,43106.375694444396,43106.375694444396,43102.0840277778,1.0,43040.0,16.0,1.05325951904188,4088.82,22.0,277.0,521.430142518076,32.0,201.400990506124,29.611418782699598,0.0
3,0.0,2.0,1.0,120.0,6929.0,0.0,0,34.8,1385.80,66.7807,27.1589,3105.69,479.931,99.0149,2424.17,50.0487,2853.19,20.1894,18.8149,262.913,2323.17,19.58,55.6293,71.9316,23.3886,99.9093,70.7424,92.0443,0.322212,1.33545,1.39247,1.16992,36.0957,3.74233,0.28292,48.222,0.650431,2782.74,124.753,1399.33,42118.2,42109.6,42069.7,2.47797,42078.6,13.3323,1.03842,3322.14,22.931,269.079,253.025,29.7971,58.9964,7.95461,127.841
4,1.0,3.0,2.0,0.0,0.0,0.0,10,0.0,0.00,66.7807,27.1589,3105.69,479.931,99.0149,2424.17,50.0487,2853.19,20.1894,18.8149,262.913,2323.17,19.58,55.6293,71.9316,23.3886,99.9093,70.7424,92.0443,0.322212,1.33545,1.39247,1.16992,36.0957,3.74233,0.28292,48.222,0.650431,2782.74,124.753,1399.33,42118.2,42109.6,42069.7,2.47797,42078.6,13.3323,1.03842,3322.14,22.931,269.079,253.025,29.7971,58.9964,7.95461,127.841
5,1.0,2.0,4.0,193.0,9744.0,277.0,15,60.7,1211.64,66.7807,27.1589,3105.69,479.931,99.0149,2424.17,50.0487,2853.19,20.1894,18.8149,262.913,2323.17,19.58,55.6293,71.9316,23.3886,99.9093,70.7424,92.0443,0.322212,1.33545,1.39247,1.16992,36.0957,3.74233,0.28292,48.222,0.650431,2782.74,124.753,1399.33,42118.2,42109.6,42069.7,2.47797,42078.6,13.3323,1.03842,3322.14,22.931,269.079,253.025,29.7971,58.9964,7.95461,127.841
6,2379,2201.0,2474.0,12,153,0,0,0.0,37.45,68.7,0.0,3147.0,800.31,125.0,2628.0,50.0,3038.01,11.0,10.1,229.2,3008.0,12.0,15.0,24.3388051948066,65.0,80.9,36.0,87.0,0.396977789751826,1.67,1.6678318186357701,1.12,21.0,0.724905444334078,0.18656210096303602,24.3388051948066,0.5367947218332321,3038.01,242.0,350.48482457454804,43099.3680555556,43099.3680555556,43064.0,0.0,43086.0,9.7,1.02552957570024,3236.4,19.0,231.0,81.74294990294129,20.1,36.2231464735487,0.0679481011955829,104.224779496054
7,1.0,1.0,1.0,264.0,39598.0,0.0,0,34.3,3599.82,66.

In [45]:
X_test_submit = np.array(test_data)

In [51]:
y_pred_submit = model.predict(X_test_submit)

y_submit = []
for i in range(len(y_pred_submit)):
    if y_pred_submit[i] > 0:
        y_submit.append(y_pred_submit[i])
        y_submit.append(y_pred_submit[i])
        y_submit.append(y_pred_submit[i])
        y_submit.append(y_pred_submit[i])
        y_submit.append(y_pred_submit[i])
        y_submit.append(y_pred_submit[i])
    else:
        y_submit.append(y_pred_submit[i]*-1)
        y_submit.append(y_pred_submit[i]*-1)
        y_submit.append(y_pred_submit[i]*-1)
        y_submit.append(y_pred_submit[i]*-1)
        y_submit.append(y_pred_submit[i]*-1)
        y_submit.append(y_pred_submit[i]*-1)
    
print(np.median(y_submit))
print(y_submit)

31.57436180114746
[7.1550727, 7.1550727, 7.1550727, 7.1550727, 7.1550727, 7.1550727, 19.791527, 19.791527, 19.791527, 19.791527, 19.791527, 19.791527, 6.679223537445068, 6.679223537445068, 6.679223537445068, 6.679223537445068, 6.679223537445068, 6.679223537445068, 155.54176, 155.54176, 155.54176, 155.54176, 155.54176, 155.54176, 7.1550727, 7.1550727, 7.1550727, 7.1550727, 7.1550727, 7.1550727, 227.62358, 227.62358, 227.62358, 227.62358, 227.62358, 227.62358, 80.31017, 80.31017, 80.31017, 80.31017, 80.31017, 80.31017, 701.3405, 701.3405, 701.3405, 701.3405, 701.3405, 701.3405, 822.39594, 822.39594, 822.39594, 822.39594, 822.39594, 822.39594, 7.1550727, 7.1550727, 7.1550727, 7.1550727, 7.1550727, 7.1550727, 7.1550727, 7.1550727, 7.1550727, 7.1550727, 7.1550727, 7.1550727, 7.1550727, 7.1550727, 7.1550727, 7.1550727, 7.1550727, 7.1550727, 7.1550727, 7.1550727, 7.1550727, 7.1550727, 7.1550727, 7.1550727, 263.99997, 263.99997, 263.99997, 263.99997, 263.99997, 263.99997, 210.20303, 210.20303,

In [52]:
counter = 0
_id = []
val = []
for i in range(len(y_submit)):
    _id.append(str(counter))
    counter += 1
    val.append(y_submit[i])
answer = pd.concat([pd.Series(_id,name='_ID_'),pd.Series(val,name=',_VAL_')],axis=1,)
answer.to_csv("submit.csv",index=False)